<a href="https://colab.research.google.com/github/prathammehta16/ML_University_Course/blob/main/20IT068_CNN_FashionMnist.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##20IT068

##Practical 8

**Importing the libraries**

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from keras.datasets import fashion_mnist
from keras.models import Sequential
from keras.layers import Dense, Conv2D, MaxPooling2D, Flatten, Dropout
from keras.utils import to_categorical
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import GridSearchCV

**Dividing the dataset in to train and test**

In [2]:
(X_train, y_train), (X_test, y_test) = fashion_mnist.load_data()


4422102/4422102 [==============================] - 0s 0us/step


**Preprocessing the data**

In [3]:
X_train = X_train.reshape(X_train.shape[0], 28, 28, 1) # Reshape input data to 4D tensor
X_test = X_test.reshape(X_test.shape[0], 28, 28, 1)

X_train = X_train.astype('float32') / 255 # Convert pixel values from integers to floats and normalize to range 0 to 1
X_test = X_test.astype('float32') / 255

**One-hot encoding the target variable**

In [4]:
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

**Creating the model**

In [10]:
def create_model(optimizer='adam'):
    model = Sequential()

    model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(28, 28, 1)))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Conv2D(64, (3, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Flatten())
    model.add(Dense(128, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(10, activation='softmax'))

    model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])
    return model

**Define hyperparameters to search over and performing grid search**

In [6]:
model = KerasClassifier(build_fn=create_model, verbose=0)

params = {'optimizer': ['adam', 'sgd'], 'batch_size': [32, 64], 'epochs': [10]}

grid = GridSearchCV(estimator=model, param_grid=params, n_jobs=-1, cv=3)
grid_result = grid.fit(X_train, y_train)

<ipython-input-6-09856fd8d978>:2: DeprecationWarning: KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  model = KerasClassifier(build_fn=create_model, verbose=0)


In [7]:
print(f"Best parameters: {grid_result.best_params_}")
print(f"Best accuracy: {grid_result.best_score_}")

Best parameters: {'batch_size': 32, 'epochs': 10, 'optimizer': 'adam'}
Best accuracy: 0.9087166587511698


**Train the model using the best hyperparameters**

In [9]:
best_model = grid_result.best_estimator_

best_model.fit(X_train, y_train)

test_acc = best_model.score(X_test, y_test)
print(f"Test accuracy: {test_acc}")

Test accuracy: 0.9107000231742859
